# EfficientNet_B4 Classifier Training

このノートブックはNFNetモデルの訓練を実行するためのものです。PyTorch Lightningを使用した学習フレームワークで、転移学習による画像分類を行います。

## Google Driveのマウント

In [ ]:
import os
import sys

# Google Colab環境かどうかを判定
IN_COLAB = 'google.colab' in str(get_ipython())

if IN_COLAB:
    print("Google Colab環境を検出しました。")
    from google.colab import drive
    drive.mount('/content/drive')
    # プロジェクトディレクトリに移動
    project_dir = '/content/drive/MyDrive/Time_Series_Classifier'
    # %cd はノートブックのセルマジックなので、os.chdirを使用
    if os.getcwd() != project_dir:
        os.chdir(project_dir)
        print(f"Moved to: {os.getcwd()}")
    # srcディレクトリをパスに追加 (main.pyと同じ階層にある場合)
    # main.pyがプロジェクトルートにあるため、srcは不要かもしれないが念のため
    src_dir = os.path.join(project_dir, 'src')
    if src_dir not in sys.path:
        sys.path.insert(0, src_dir)
    # configsディレクトリもパスに追加 (config_utilsのため)
    configs_dir = os.path.join(project_dir, 'configs')
    if configs_dir not in sys.path:
        sys.path.insert(0, configs_dir)
else:
    print("ローカル環境を検出しました。")
    # ローカルのプロジェクトディレクトリを設定
    project_dir = 'i:/Efficient_Net_Classifier'
    # カレントディレクトリがプロジェクトディレクトリでない場合は移動
    if os.getcwd() != os.path.abspath(project_dir):
        os.chdir(project_dir)
        print(f"Moved to: {os.getcwd()}")
    # srcディレクトリをパスに追加
    src_dir = os.path.join(project_dir, 'src')
    if src_dir not in sys.path:
        sys.path.insert(0, src_dir)
    # configsディレクトリもパスに追加
    configs_dir = os.path.join(project_dir, 'configs')
    if configs_dir not in sys.path:
        sys.path.insert(0, configs_dir)

print(f"現在のディレクトリ: {os.getcwd()}")
print(f"プロジェクトディレクトリ: {project_dir}")
print(f"Pythonパスにsrcを追加: {src_dir in sys.path}")
print(f"Pythonパスにconfigsを追加: {configs_dir in sys.path}")

## 必要なライブラリのインストール

In [ ]:
# requirements.txtからインストール
# !pip install torch torchvision pytorch-lightning torchmetrics PyYAML scikit-learn pandas
! pip install lightning torchmetrics timm seaborn

## GPUの確認

In [ ]:
!nvidia-smi

import torch
print(f"CUDA利用可能: {torch.cuda.is_available()}")
print(f"利用可能なGPU数: {torch.cuda.device_count()}")
if torch.cuda.is_available():
    print(f"現在のGPU: {torch.cuda.get_device_name(0)}")

## データセットの確認（オプション）

In [ ]:
# # データセットの構造確認（オプション）
# !ls -la /content/drive/MyDrive/Time_Series_Classifier/data/dataset_a_15m_winsize40/train
# # 各クラスの画像数を確認
# !find /content/drive/MyDrive/Time_Series_Classifier/data/dataset_a_15m_winsize40/train -type f | grep -v "/__" | sort | cut -d/ -f8 | uniq -c
# !find /content/drive/MyDrive/Time_Series_Classifier/data/dataset_a_15m_winsize40/test -type f | grep -v "/__" | sort | cut -d/ -f8 | uniq -c

## 設定ファイルの確認と編集（必要に応じて）

In [ ]:
# 設定ファイルの内容確認
import yaml
import os
import platform

# 環境に応じた設定ファイルパスを設定
if IN_COLAB:
    config_filename = 'config_for_google_colab.yaml'
else:
    config_filename = 'config.yaml' # ローカル用の設定ファイル

config_path = os.path.join(project_dir, 'configs', config_filename)
print(f"使用する設定ファイル: {config_path}")

# 設定ファイルの内容確認
try:
    with open(config_path, 'r', encoding='utf-8') as f:
        print("--- 設定ファイル内容 ---")
        print(f.read())
        print("----------------------")
except FileNotFoundError:
    print(f"エラー: 設定ファイル {config_path} が見つかりません。パスを確認してください。")
# 広範な例外捕捉は避ける (例: yaml.YAMLError など、より具体的な例外を捕捉する)
except yaml.YAMLError as e:
    print(f"設定ファイルの解析中にエラーが発生しました: {e}")
except IOError as e:
    print(f"設定ファイルの読み込み中にI/Oエラーが発生しました: {e}")

# --- 注意 ---
# 設定の変更は直接YAMLファイルを編集するか、main.py側で行います。
# このノートブックでは設定の読み込み確認のみを行います。

## Windows環境での注意事項

Windows環境では、Pythonのマルチプロセッシングの仕組み上の制約から、`num_workers` を0以外に設定するとエラーが発生しやすくなります。これは主に以下の理由によります：

- **プロセス生成方法の違い**: LinuxなどのUnix系OSでは、フォーク（`fork`）システムコールを使ってプロセスを生成するため、親プロセスの状態をそのままコピーできます。一方、Windowsでは `spawn` メソッドが使われます。`spawn` は新しいプロセスを最初から初期化するため、親プロセス上で定義された状態やグローバル変数が継承されず、必要な初期化手順を踏む必要があります。

- **`if __name__ == "__main__":` の重要性**: Windowsでは、コードが必ずこのブロック内で実行されるように構成する必要があります。

- **Jupyter環境の制約**: 特にJupyter環境でのマルチプロセスはWindows上で問題を起こしやすいです。

このノートブックでは、Windows環境を自動検出して `num_workers=0` に設定するようにしています。**パフォーマンスを最大化するには、Google Colab環境での実行を推奨します。**

## 訓練スクリプトの実行

In [ ]:
# main.py を実行して訓練を開始
# main.py は内部で環境を判断し、適切な設定ファイルを読み込みます
print(f"プロジェクトディレクトリ ({project_dir}) で main.py を実行します...")
# !python main.py コマンドを実行
# ノートブック環境からPythonスクリプトを実行する場合、カレントディレクトリに注意
# 上のセルで os.chdir を使ってプロジェクトディレクトリに移動済みのはず
! python main.py

## TensorBoardによる訓練の可視化

In [ ]:
import os
# config_utils.py が configs ディレクトリにあることを確認
try:
    # src と configs が sys.path にあるため、直接インポート
    from config_utils import load_config # config読み込み用ユーティリティ
except ImportError:
    print("エラー: config_utils が見つかりません。Pythonパスを確認してください。")
    # 必要であればパスを再度追加
    configs_dir = os.path.join(project_dir, 'configs')
    if configs_dir not in sys.path:
        sys.path.insert(0, configs_dir)
        print(f"'{configs_dir}' をPythonパスに追加しました。")
        try:
            from config_utils import load_config
        except ImportError as ie:
             print(f"再試行しましたが、config_utils のインポートに失敗しました: {ie}")
             # ここで処理を中断するか、デフォルトパスを使うなどの代替策を検討
             raise ie # エラーを再発生させる

# 設定ファイルを再度読み込み、ログディレクトリを取得
try:
    # config_path が前のセルで定義されていることを確認
    if 'config_path' not in globals():
        raise NameError("'config_path' is not defined. Please run the cell defining it.")
    config = load_config(config_path)
    # log_dir_tb = config.get("logs_dir", os.path.join(project_dir, "logs")) # 設定ファイルから取得する代わりに、期待されるパスを直接構築
    experiment_name = "stock_classifier" # main.pyでの設定と合わせる
    # 正しいログディレクトリのベースパスを構築
    correct_log_base_dir = os.path.join(project_dir, "logs")
    tensorboard_logdir = os.path.join(correct_log_base_dir, experiment_name) # 常に logs/stock_classifier を指すように修正

    # Colab用のTensorBoard拡張を読み込む
    if IN_COLAB:
        %load_ext tensorboard
        print(f"TensorBoard ログディレクトリ: {tensorboard_logdir}")
        # パスにスペースが含まれる可能性を考慮して引用符で囲む
        %tensorboard --logdir="{tensorboard_logdir}"
    else:
        print("ローカル環境です。TensorBoardを手動で起動してください:")
        # Windowsの場合、パスをダブルクォーテーションで囲むのが一般的
        print(f"tensorboard --logdir=\"{tensorboard_logdir}\"")
except FileNotFoundError:
    print(f"エラー: 設定ファイル {config_path} が見つかりません。")
except NameError as e:
    print(f"エラー: 必要な変数(config_pathなど)が定義されていません - {e}")
except KeyError as e:
    print(f"エラー: 設定ファイルに必要なキー ('logs_dir'など) がありません - {e}")
# 広範な例外捕捉は避ける
except Exception as e: # より具体的な例外を捕捉することが望ましい
    print(f"TensorBoardの準備中に予期せぬエラーが発生しました: {e}")
    import traceback
    traceback.print_exc()

In [ ]:
# 新しいセルに追加するコード
import os
import re
import sys
import yaml # yaml をインポート

# config_utils が見つからない場合に備えてパスを追加
# project_dir は ID '9f53d608' のセルで定義されている想定
try:
    from configs.config_utils import load_config, get_project_root
except ImportError:
    print("config_utils が見つかりません。パスを確認・追加します。")
    configs_dir_util = os.path.join(project_dir, 'configs')
    if configs_dir_util not in sys.path:
        sys.path.insert(0, configs_dir_util)
        print(f"'{configs_dir_util}' をPythonパスに追加しました。")
        try:
            from configs.config_utils import load_config, get_project_root
        except ImportError as ie_util:
            print(f"再試行しましたが、config_utils のインポートに失敗しました: {ie_util}")
            raise ie_util # エラーを再発生

# find_best_checkpoint 関数の定義 (evaluate.py/visualize.py と同じもの)
def find_best_checkpoint(config, metric="f1"):
    """
    指定されたメトリックに基づいて最適なチェックポイントファイルを見つける。
    新しいディレクトリ構造とファイル名形式に対応。

    Args:
        config (dict): 設定辞書。'model_mode', 'model_architecture_name' を含む。
        metric (str): 最適化するメトリック ('f1' または 'loss')。

    Returns:
        str or None: 最適なチェックポイントファイルのパス。見つからない場合はNone。
    """
    # project_root は get_project_root() で取得するか、ノートブックの project_dir を使う
    # ここでは get_project_root() を使う
    try:
        project_root = get_project_root()
    except Exception as e_proj_root:
         print(f"get_project_root() でエラー: {e_proj_root}. ノートブックの 'project_dir' を使用します。")
         # project_dir がグローバルスコープにあることを期待
         if 'project_dir' not in globals():
              print("エラー: 'project_dir' 変数が定義されていません。")
              return None
         project_root = project_dir

    model_mode = config.get("model_mode", "single")
    model_architecture_name = config.get("model_architecture_name", "default_model")
    # 新しいチェックポイントディレクトリパスを構築
    # config から checkpoint_dir を取得し、その下に model_mode/model_architecture_name を追加
    base_checkpoint_dir = config.get("checkpoint_dir", os.path.join(project_root, "checkpoints"))
    checkpoint_dir = os.path.join(base_checkpoint_dir, model_mode, model_architecture_name)

    print(f"チェックポイントディレクトリを検索中: {checkpoint_dir}")

    if not os.path.isdir(checkpoint_dir):
        print(f"エラー: チェックポイントディレクトリが見つかりません: {checkpoint_dir}")
        # 設定ファイルで指定された base_checkpoint_dir も確認
        if base_checkpoint_dir != checkpoint_dir and os.path.isdir(base_checkpoint_dir):
             print(f"警告: サブディレクトリ '{model_mode}/{model_architecture_name}' はありませんが、ベースディレクトリ '{base_checkpoint_dir}' は存在します。")
             # ベースディレクトリ内も検索するかどうか？ -> ここではしない
        return None

    # 新しいファイル名形式 'epoch={epoch:05d}-val_loss={val_loss:.4f}-val_f1={val_f1:.4f}.ckpt'
    # metric に応じた正規表現パターン
    if metric == "f1":
        # val_f1 を抽出するパターン
        pattern = re.compile(r'epoch=(\d+)-val_loss=([\d.]+)-val_f1=([\d.]+)\.ckpt')
        metric_index = 2 # F1スコアは3番目のキャプチャグループ (0-based index)
        best_metric_val = -1.0 # F1スコアは高いほど良い
        compare_func = lambda current, best: current > best
    elif metric == "loss":
        # val_loss を抽出するパターン
        pattern = re.compile(r'epoch=(\d+)-val_loss=([\d.]+)-val_f1=([\d.]+)\.ckpt')
        metric_index = 1 # 損失は2番目のキャプチャグループ
        best_metric_val = float('inf') # 損失は低いほど良い
        compare_func = lambda current, best: current < best
    else:
        print(f"エラー: 未知のメトリック '{metric}'。'f1' または 'loss' を使用してください。")
        return None

    best_checkpoint_path = None
    found_checkpoints = []
    last_ckpt_path = None # last.ckpt のパスを初期化

    try:
        for filename in os.listdir(checkpoint_dir):
            match = pattern.match(filename)
            if match:
                found_checkpoints.append(filename)
                try:
                    # 指定されたメトリックの値を抽出
                    current_metric_val = float(match.group(metric_index + 1)) # グループインデックスは1から始まるため+1
                    print(f"  チェックポイント '{filename}' の {metric}: {current_metric_val}")
                    # 最良のメトリック値を更新
                    if compare_func(current_metric_val, best_metric_val):
                        best_metric_val = current_metric_val
                        best_checkpoint_path = os.path.join(checkpoint_dir, filename)
                except (ValueError, IndexError) as e:
                    print(f"  警告: ファイル '{filename}' のメトリック値の解析中にエラー: {e}")
                    continue
            # 'last.ckpt' も候補として保持 (最良が見つからない場合に使用)
            elif filename == "last.ckpt":
                 last_ckpt_path = os.path.join(checkpoint_dir, filename)

    except OSError as e:
        print(f"エラー: チェックポイントディレクトリの読み取り中にエラーが発生しました: {e}")
        return None

    if best_checkpoint_path:
        print(f"最適な {metric} ({best_metric_val:.4f}) を持つチェックポイントが見つかりました: {best_checkpoint_path}")
        return best_checkpoint_path
    elif last_ckpt_path and os.path.exists(last_ckpt_path): # last_ckpt_path が None でないことを確認
         print(f"警告: 最適な {metric} を持つチェックポイントが見つかりませんでした。'last.ckpt' を使用します: {last_ckpt_path}")
         return last_ckpt_path
    else:
        print(f"警告: 有効なチェックポイントファイルがディレクトリ '{checkpoint_dir}' に見つかりませんでした。")
        # last.ckpt も見つからなかった場合、古い形式のチェックポイントをベースディレクトリで探す試み（オプション）
        print(f"念のため、ベースディレクトリ '{base_checkpoint_dir}' で古い形式の 'last.ckpt' を探します...")
        old_last_ckpt = os.path.join(base_checkpoint_dir, 'last.ckpt')
        if os.path.exists(old_last_ckpt):
             print(f"警告: ベースディレクトリで古い形式の 'last.ckpt' を見つけました。これを使用します: {old_last_ckpt}")
             return old_last_ckpt
        else:
             print(f"警告: ベースディレクトリにも 'last.ckpt' が見つかりませんでした。")
             return None

print("find_best_checkpoint 関数が定義されました。")

## 学習済みモデルのテストと評価

In [ ]:
# 学習後、最良のモデルを使って評価を実行するには
# すでにテストは訓練時に実行されているはずですが、個別に実行したい場合は以下を利用

import os
import sys
import torch
import lightning.pytorch as pl # lightning に変更
import re

# モジュールパスの確認 (必要であれば)
# 最初のセルで追加済みのはずだが、念のため確認・追加
project_root_dir_test = os.path.abspath(os.path.join(os.getcwd())) # main.pyと同じ階層を想定
src_dir_test = os.path.join(project_root_dir_test, 'src')
configs_dir_test = os.path.join(project_root_dir_test, 'configs')

if src_dir_test not in sys.path:
    sys.path.insert(0, src_dir_test)
if configs_dir_test not in sys.path:
     sys.path.insert(0, configs_dir_test)

try:
    # src と configs が sys.path にあるため、直接インポート
    from model import StockClassifier
    from datamodule import StockDataModule
    from config_utils import load_config
except ImportError as e:
    print(f"必要なモジュールのインポートに失敗しました: {e}")
    print("モジュールパスを確認してください。sys.path:", sys.path)
    # インポート失敗時は処理を中断
    raise e

try:
    # 設定ファイルの読み込み（訓練時と同じものを使用）
    # config_path は前のセルで定義されている想定
    if 'config_path' not in globals():
        # config_path が未定義の場合、環境に応じて再設定
        if IN_COLAB:
            config_filename_test = 'config_for_google_colab.yaml'
        else:
            config_filename_test = 'config.yaml'
        config_path = os.path.join(project_root_dir_test, 'configs', config_filename_test)
        print(f"警告: 'config_path' が未定義でした。'{config_path}' を使用します。")

    config = load_config(config_path)

    # 最良モデルのチェックポイントパスを取得
    # configからcheckpoint_dirを取得、なければデフォルトパス
    checkpoint_dir_path = config.get("checkpoint_dir", os.path.join(project_root_dir_test, "checkpoints"))
    print(f"チェックポイントディレクトリ: {checkpoint_dir_path}")

    # ModelCheckpointで設定したファイル名パターンに基づいて探す
    # val_f1 が最高のモデルを探す (mode='max')
    # ファイル名形式: 'model_epoch_{epoch:05d}_val_loss_{val_loss:.4f}_val_f1_{val_f1:.4f}.ckpt'
    checkpoints = []
    if os.path.isdir(checkpoint_dir_path):
        checkpoints = [f for f in os.listdir(checkpoint_dir_path) if f.startswith("model_epoch_") and "val_f1_" in f and f.endswith(".ckpt")]
    else:
        print(f"エラー: チェックポイントディレクトリが見つかりません: {checkpoint_dir_path}")

    best_checkpoint_path = None
    best_f1 = -1.0

    if checkpoints:
        print(f"見つかったチェックポイント候補: {checkpoints}")
        # 正規表現を使用してファイル名からval_f1の値を抽出
        f1_pattern = re.compile(r'val_f1_([0-9.]+)')
        for fname in checkpoints:
            try:
                # 正規表現でF1スコアを抽出 (例: model_epoch_00011_val_loss_0.9229_val_f1_0.6907.ckpt → 0.6907)
                f1_match = f1_pattern.search(fname)
                if f1_match:
                    current_f1 = float(f1_match.group(1))
                    print(f"  チェックポイント '{fname}' の F1スコア: {current_f1}")
                    if current_f1 > best_f1:
                        best_f1 = current_f1
                        best_checkpoint_path = os.path.join(checkpoint_dir_path, fname)
                else:
                    print(f"  警告: ファイル '{fname}' からF1スコアを抽出できませんでした (パターンに一致しない)")
            except (ValueError, AttributeError) as parse_err:
                print(f"  警告: ファイル '{fname}' の解析中にエラーが発生しました: {parse_err}")
                continue # 次のファイルへ
        if best_checkpoint_path:
             print(f"最高のF1スコア ({best_f1:.4f}) を持つチェックポイントが見つかりました: {best_checkpoint_path}")
        else:
             print("F1スコアを含む有効なチェックポイントが見つかりませんでした。")

    # 最良が見つからない場合は last.ckpt を試す
    if best_checkpoint_path is None:
        last_ckpt = os.path.join(checkpoint_dir_path, 'last.ckpt')
        if os.path.exists(last_ckpt):
            best_checkpoint_path = last_ckpt
            print(f"最良のF1チェックポイントが見つかりません。最新のチェックポイントを使用します: {best_checkpoint_path}")
        else:
             # last.ckpt も見つからない場合
             print(f"エラー: 評価に使用できるチェックポイント ('model_e...val_f1...' または 'last.ckpt') がディレクトリ '{checkpoint_dir_path}' に見つかりません。")
             # エラーにするか、Noneのまま進むかは要件次第
             # ここでは None のまま進み、後続のifで処理する

    if best_checkpoint_path:
        print(f"評価に使用するチェックポイント: {best_checkpoint_path}")
        # データモジュールの準備（テスト用）
        data_module = StockDataModule(config)
        # data_module.setup("test") # testメソッド内で自動的に呼ばれる

        # モデルのロード
        # configを渡して、チェックポイント保存時と異なる可能性のある設定に対応
        # strict=False は、モデル構造が変わっていない限り、一部の不一致を許容する
        try:
            model = StockClassifier.load_from_checkpoint(best_checkpoint_path, config=config, strict=False)
            print("モデルのロードに成功しました。")
        except FileNotFoundError as model_load_err:
            print(f"エラー: チェックポイントファイルが見つかりません: {model_load_err}")
            raise model_load_err
        except Exception as model_load_err: # より具体的な例外捕捉が望ましい
            print(f"モデルのロード中にエラーが発生しました: {model_load_err}")
            raise model_load_err

        # テスト用トレーナーの設定
        # accelerator と devices を config や環境に合わせて設定
        accelerator_setting = "auto"
        devices_setting = "auto"
        if config.get("force_gpu", False) and torch.cuda.is_available():
            accelerator_setting = "gpu"
            devices_setting = 1 # テストは通常1デバイス
        elif config.get("force_cpu", False):
            accelerator_setting = "cpu"
            devices_setting = 1

        tester = pl.Trainer(
            accelerator=accelerator_setting,
            devices=devices_setting,
            logger=False, # テスト結果はログ不要
            precision=config.get('precision', '32-true') # 訓練時と同じ精度を使用
        )

        # テストの実行
        print("\nテストを再実行します...")
        try:
            test_results = tester.test(model, datamodule=data_module)
            print("\nテスト結果:")
            print(test_results)
        except Exception as test_err: # より具体的な例外捕捉が望ましい
             print(f"テストの実行中にエラーが発生しました: {test_err}")
             import traceback
             traceback.print_exc()

    else:
        # best_checkpoint_path が None の場合（チェックポイントが見つからなかった場合）
        print("テストを実行できませんでした。有効なチェックポイントが見つかりません。")

# FileNotFoundError は設定ファイル読み込み時に発生する可能性
# KeyError は config 辞書に必要なキーがない場合に発生する可能性
# NameError は config_path など、前のセルで定義されるべき変数が未定義の場合
# ImportError はモジュールインポート失敗時
except (FileNotFoundError, KeyError, NameError, ImportError) as e:
    print(f"エラーが発生しました: {e}")
# 広範な例外捕捉は避ける
except Exception as e: # より具体的な例外を捕捉することが望ましい
    print(f"テスト準備または実行中に予期せぬエラーが発生しました: {e}")
    import traceback
    traceback.print_exc()

## モデル予測の可視化（オプション）

In [ ]:
# 必要な変数が定義されているか確認し、なければ再定義/ロード
import os
import sys
import torch
import re

# モジュールパスの確認 (必要であれば)
# 最初のセルで追加済みのはずだが、念のため確認・追加
project_root_dir_vis = os.path.abspath(os.path.join(os.getcwd()))
src_dir_vis = os.path.join(project_root_dir_vis, 'src')
configs_dir_vis = os.path.join(project_root_dir_vis, 'configs')

if src_dir_vis not in sys.path:
    sys.path.insert(0, src_dir_vis)
if configs_dir_vis not in sys.path:
     sys.path.insert(0, configs_dir_vis)

# グローバルスコープに変数が存在するかチェック
# 存在しない場合のみ再ロードを試みる
if 'data_module' not in globals() or 'model' not in globals():
    print("data_module または model が未定義です。再定義/再ロードを試みます...")
    try:
        # src と configs が sys.path にあるため、直接インポート
        from model import StockClassifier
        from datamodule import StockDataModule
        from config_utils import load_config

        # 設定ファイルの読み込み
        # config_path は前のセルで定義されている想定
        if 'config_path' not in globals():
            # config_path が未定義の場合、環境に応じて再設定
            if IN_COLAB:
                config_filename_vis = 'config_for_google_colab.yaml'
            else:
                config_filename_vis = 'config.yaml'
            config_path = os.path.join(project_root_dir_vis, 'configs', config_filename_vis)
            print(f"警告: 'config_path' が未定義でした。'{config_path}' を使用します。")

        config = load_config(config_path)
        # データモジュールの準備
        data_module = StockDataModule(config)
        # 可視化にはテストデータを使うことが多いので 'test' を指定
        # setup() は dataloader() 呼び出し時に内部で実行される場合もあるが、明示的に呼ぶ
        data_module.setup('test')
        print("DataModuleをセットアップしました (stage='test')。")

        # モデルのロード（前のセルで特定した最良または最新のチェックポイントから）
        # best_checkpoint_path が前のセルで定義されていることを期待
        if 'best_checkpoint_path' in globals() and best_checkpoint_path and os.path.exists(best_checkpoint_path):
             print(f"前のセルで特定されたチェックポイントを使用します: {best_checkpoint_path}")
             model = StockClassifier.load_from_checkpoint(best_checkpoint_path, config=config, strict=False)
             print(f"モデルを {best_checkpoint_path} からロードしました。")
        else:
             # best_checkpoint_path が未定義または無効な場合、再度探す
             print("警告: 'best_checkpoint_path' が未定義または無効です。再度チェックポイントを探します...")
             checkpoint_dir_path_vis = config.get("checkpoint_dir", os.path.join(project_root_dir_vis, "checkpoints"))
             # 前のセルと同様のロジックで最良チェックポイントを探す
             checkpoints_vis = []
             if os.path.isdir(checkpoint_dir_path_vis):
                 checkpoints_vis = [f for f in os.listdir(checkpoint_dir_path_vis) if f.startswith("model_epoch_") and "val_f1_" in f and f.endswith(".ckpt")]
             temp_best_path = None
             temp_best_f1 = -1.0
             if checkpoints_vis:
                 print(f"見つかったチェックポイント候補: {checkpoints_vis}")
                 # 正規表現を使用してファイル名からval_f1の値を抽出
                 f1_pattern = re.compile(r'val_f1_([0-9.]+)')
                 for fname in checkpoints_vis:
                     try:
                         # 正規表現でF1スコアを抽出 (例: model_epoch_00011_val_loss_0.9229_val_f1_0.6907.ckpt → 0.6907)
                         f1_match = f1_pattern.search(fname)
                         if f1_match:
                             current_f1 = float(f1_match.group(1))
                             print(f"  チェックポイント '{fname}' の F1スコア: {current_f1}")
                             if current_f1 > temp_best_f1:
                                 temp_best_f1 = current_f1
                                 temp_best_path = os.path.join(checkpoint_dir_path_vis, fname)
                         else:
                             print(f"  警告: ファイル '{fname}' からF1スコアを抽出できませんでした (パターンに一致しない)")
                     except (ValueError, AttributeError) as parse_err:
                         print(f"  警告: ファイル '{fname}' の解析中にエラーが発生しました: {parse_err}")
                         continue # 次のファイルへ
             if temp_best_path:
                 best_checkpoint_path = temp_best_path # グローバル変数も更新
                 print(f"最高のF1スコア ({temp_best_f1:.4f}) を持つチェックポイントを再検出しました: {best_checkpoint_path}")
                 model = StockClassifier.load_from_checkpoint(best_checkpoint_path, config=config, strict=False)
                 print(f"モデルを {best_checkpoint_path} からロードしました。")
             else:
                 last_ckpt_vis = os.path.join(checkpoint_dir_path_vis, 'last.ckpt')
                 if os.path.exists(last_ckpt_vis):
                     best_checkpoint_path = last_ckpt_vis # グローバル変数も更新
                     print(f"最良が見つからず、最新のモデルを使用します: {best_checkpoint_path}")
                     model = StockClassifier.load_from_checkpoint(best_checkpoint_path, config=config, strict=False)
                     print(f"モデルを {best_checkpoint_path} からロードしました。")
                 else:
                     print(f"エラー: ロードするモデルのチェックポイントがディレクトリ '{checkpoint_dir_path_vis}' に見つかりません。")
                     raise FileNotFoundError(f"チェックポイントが見つかりません in {checkpoint_dir_path_vis}")

    # ImportError はモジュールが見つからない場合
    # FileNotFoundError は設定ファイルやチェックポイントが見つからない場合
    # NameError は config_path などが未定義の場合
    except (ImportError, FileNotFoundError, NameError) as e:
        print(f"エラーが発生しました: {e}")
        # エラー発生時は以降のセルが実行できないため、再発生させる
        raise e
    # 広範な例外捕捉は避ける
    except Exception as e: # より具体的な例外を捕捉することが望ましい
        print(f"可視化のための再定義/再ロード中に予期せぬエラーが発生しました: {e}")
        import traceback
        traceback.print_exc()
        # エラー発生時は以降のセルが実行できないため、再発生させる
        raise e
else:
    print("data_module と model は既に定義済みです。")

In [ ]:
# テストデータから数サンプルを選び、予測結果を可視化する
import matplotlib.pyplot as plt
import numpy as np
import torch
from torchvision import transforms
import random
import sys
import os

try:
    # 設定ファイルからクラス名と正規化パラメータを取得
    # config がロードされていることを確認
    if 'config' not in globals():
        # config がなければロードを試みる
        print("警告: 'config' が未定義です。ロードを試みます...")
        configs_dir_vis2 = os.path.join(project_dir, 'configs') # project_dir は最初のセルで定義済みのはず
        if configs_dir_vis2 not in sys.path:
             sys.path.insert(0, configs_dir_vis2)
        try:
            from config_utils import load_config
        except ImportError as ie:
             print(f"config_utils のインポートに失敗しました: {ie}")
             raise ie
        if 'config_path' not in globals():
             # config_path が未定義の場合、環境に応じて再設定
             if IN_COLAB:
                 config_filename_vis2 = 'config_for_google_colab.yaml'
             else:
                 config_filename_vis2 = 'config.yaml'
             config_path = os.path.join(project_dir, 'configs', config_filename_vis2)
             print(f"警告: 'config_path' が未定義でした。'{config_path}' を使用します。")
        try:
            config = load_config(config_path)
        except FileNotFoundError as fe:
            print(f"設定ファイルが見つかりません: {fe}")
            raise fe
        except yaml.YAMLError as ye:
            print(f"設定ファイルの解析エラー: {ye}")
            raise ye

    # config からクラス名、平均、標準偏差を取得、なければデフォルト値
    class_names = config.get('class_names', ['Class 0', 'Class 1', 'Class 2'])
    mean = config.get('dataset_mean', [0.485, 0.456, 0.406])
    std = config.get('dataset_std', [0.229, 0.224, 0.225])
    print(f"クラス名: {class_names}")
    print(f"データセット平均: {mean}, 標準偏差: {std}")

    # データローダーからランダムに1バッチ取得
    # data_module がロードされていることを確認
    if 'data_module' not in globals():
         raise NameError("data_module が定義されていません。前のセルを実行してください。")
    try:
        test_loader = data_module.test_dataloader()
        images, labels = next(iter(test_loader))
        print(f"テストローダーからバッチを取得しました。画像形状: {images.shape}, ラベル形状: {labels.shape}")
    except StopIteration:
        print("エラー: テストデータローダーが空です。")
        # データがない場合は処理を中断
        raise StopIteration("テストデータがありません")
    except Exception as dl_err: # DataLoaderに関する他のエラー
        print(f"データローダーからのバッチ取得中にエラー: {dl_err}")
        raise dl_err

    # モデルを評価モードに設定
    # model がロードされていることを確認
    if 'model' not in globals():
        raise NameError("model が定義されていません。前のセルを実行してください。")
    model.eval()
    print("モデルを評価モードに設定しました。")

    # GPUが利用可能ならモデルとデータをGPUへ移動
    # config から force_gpu を取得、なければデフォルト False
    use_gpu = config.get('force_gpu', False) and torch.cuda.is_available()
    device = torch.device("cuda" if use_gpu else "cpu")
    print(f"使用デバイス: {device}")
    try:
        model.to(device)
        images = images.to(device)
    except Exception as device_err:
        print(f"モデルまたはデータのデバイス転送中にエラー: {device_err}")
        raise device_err

    # 予測実行
    print("予測を実行します...")
    with torch.no_grad():
        try:
            # --- 修正箇所 ---
            # model(images) の戻り値が logits のみであると仮定して修正
            logits = model(images)
            # reasoning_soft は削除
            # --- 修正箇所ここまで ---

            # 結果をCPUに戻してから計算（メモリ節約とNumPy変換のため）
            logits = logits.cpu()
            probs = torch.nn.functional.softmax(logits, dim=1)
            preds = torch.argmax(logits, dim=1)
            print("予測が完了しました。")
        except Exception as pred_err:
            print(f"モデルのフォワードパス実行中にエラー: {pred_err}")
            raise pred_err # エラーを再発生させてトレースバックを表示

    # 結果の可視化
    print("結果を可視化します...")
    # 正規化解除のための変換
    try:
        inv_normalize = transforms.Normalize(
            mean=[-m/s for m, s in zip(mean, std)],
            std=[1/s for s in std]
        )
    except ZeroDivisionError:
        print("エラー: 標準偏差にゼロが含まれています。設定ファイルを確認してください。")
        raise ZeroDivisionError("標準偏差がゼロです")

    # 表示する画像数を決定 (最大8枚)
    num_images_to_show = min(8, len(images))
    if num_images_to_show == 0:
        print("表示する画像がありません。")
    else:
        # 描画領域のサイズ調整 (4列表示を想定)
        num_rows = (num_images_to_show + 3) // 4
        plt.figure(figsize=(16, 4 * num_rows)) # 横幅を少し広げる

        # バッチの先頭から表示
        indices = range(num_images_to_show)

        for i, idx in enumerate(indices):
            plt.subplot(num_rows, 4, i + 1)
            # 画像をCPUに戻し、正規化を解除して表示用に次元を並び替え
            img_tensor = images[idx].cpu() # 元のテンソルをCPUへ
            try:
                img = inv_normalize(img_tensor).permute(1, 2, 0).numpy()
            except Exception as norm_err:
                print(f"画像の正規化解除または次元並び替え中にエラー: {norm_err}")
                # エラーが発生した画像はスキップ
                plt.title("表示エラー")
                plt.axis('off')
                continue

            # 値を0-1の範囲にクリップ (正規化解除で範囲外になる可能性)
            img = np.clip(img, 0, 1)
            plt.imshow(img)

            # ラベルと予測を取得
            true_label_idx = labels[idx].item()
            pred_label_idx = preds[idx].item()

            # クラス名リスト外のインデックスアクセスを防ぐ
            true_label_name = class_names[true_label_idx] if 0 <= true_label_idx < len(class_names) else f"Unknown({true_label_idx})"
            pred_label_name = class_names[pred_label_idx] if 0 <= pred_label_idx < len(class_names) else f"Unknown({pred_label_idx})"

            # タイトルに真ラベル、予測ラベル、予測確率を表示
            plt.title(f"True: {true_label_name}\nPred: {pred_label_name} (Prob: {probs[idx][pred_label_idx]:.2f})")
            plt.axis('off') # 軸を非表示に

        plt.tight_layout() # サブプロット間のスペースを調整
        plt.show()
        print("可視化が完了しました。")

# NameError は model, data_module, config_path, config が未定義の場合
# FileNotFoundError は設定ファイルが見つからない場合（configロード時）
# KeyError は config 辞書に必要なキーがない場合
# AttributeError はオブジェクトに必要な属性がない場合（例: model.eval()）
# StopIteration はデータローダーが空の場合
# ZeroDivisionError は標準偏差がゼロの場合
except (NameError, FileNotFoundError, KeyError, AttributeError, StopIteration, ZeroDivisionError) as e:
     print(f"可視化に必要な変数、設定、またはデータが見つからないか、アクセスできませんでした: {e}")
# 広範な例外捕捉は避ける
except Exception as e: # より具体的な例外を捕捉することが望ましい
    print(f"予測結果の可視化中に予期せぬエラーが発生しました: {e}")
    import traceback
    traceback.print_exc()

## 混同行列の可視化

In [ ]:
# テストデータセット全体での混同行列を表示
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import torch
import numpy as np
import sys
import os

try:
    # 設定ファイルからクラス名を取得
    # config がロードされていることを確認
    if 'config' not in globals():
        # config がなければロードを試みる
        print("警告: 'config' が未定義です。ロードを試みます...")
        configs_dir_cm = os.path.join(project_dir, 'configs') # project_dir は最初のセルで定義済みのはず
        if configs_dir_cm not in sys.path:
             sys.path.insert(0, configs_dir_cm)
        try:
            from config_utils import load_config
        except ImportError as ie:
             print(f"config_utils のインポートに失敗しました: {ie}")
             raise ie
        if 'config_path' not in globals():
             # config_path が未定義の場合、環境に応じて再設定
             if IN_COLAB:
                 config_filename_cm = 'config_for_google_colab.yaml'
             else:
                 config_filename_cm = 'config.yaml'
             config_path = os.path.join(project_dir, 'configs', config_filename_cm)
             print(f"警告: 'config_path' が未定義でした。'{config_path}' を使用します。")
        try:
            config = load_config(config_path)
        except FileNotFoundError as fe:
            print(f"設定ファイルが見つかりません: {fe}")
            raise fe
        except yaml.YAMLError as ye:
            print(f"設定ファイルの解析エラー: {ye}")
            raise ye

    # config からクラス名を取得、なければデフォルト値
    class_names = config.get('class_names', ['Sell', 'Buy', 'Hold'])
    print(f"クラス名: {class_names}")

    # すべてのテストデータでの予測を収集するためのリスト
    true_labels = []
    pred_labels = []

    # モデルを評価モードに
    # model がロードされていることを確認
    if 'model' not in globals():
        raise NameError("model が定義されていません。前のセルを実行してください。")
    model.eval()
    print("モデルを評価モードに設定しました。")

    # GPUが利用可能ならモデルをGPUへ
    # config から force_gpu を取得、なければデフォルト False
    use_gpu_cm = config.get('force_gpu', False) and torch.cuda.is_available()
    device_cm = torch.device("cuda" if use_gpu_cm else "cpu")
    print(f"使用デバイス: {device_cm}")
    try:
        model.to(device_cm)
    except Exception as device_err_cm:
        print(f"モデルのデバイス転送中にエラー: {device_err_cm}")
        raise device_err_cm

    print("テストデータ全体で予測を収集しています...")
    # data_module がロードされていることを確認
    if 'data_module' not in globals():
        raise NameError("data_module が定義されていません。前のセルを実行してください。")

    try:
        test_loader_cm = data_module.test_dataloader()
        with torch.no_grad():
            for batch in test_loader_cm:
                images, labels = batch
                # データを適切なデバイスへ
                try:
                    images = images.to(device_cm)
                except Exception as batch_device_err:
                    print(f"バッチデータのデバイス転送中にエラー: {batch_device_err}")
                    # このバッチをスキップするか、エラーを発生させるか検討
                    continue # スキップする場合

                # 予測実行
                try:
                    # --- 修正箇所 ---
                    # model(images) の戻り値が logits のみであると仮定して修正
                    logits = model(images) # reasoning_soft は削除
                    # --- 修正箇所ここまで ---
                    preds = torch.argmax(logits, dim=1)
                except Exception as batch_pred_err:
                    print(f"バッチ予測中にエラー: {batch_pred_err}")
                    # このバッチをスキップするか、エラーを発生させるか検討
                    continue # スキップする場合

                # 結果をCPUに集める (NumPy変換のため)
                true_labels.extend(labels.cpu().numpy())
                pred_labels.extend(preds.cpu().numpy())
    except StopIteration:
         print("警告: テストデータローダーが空でした。混同行列は計算できません。")
         # データがない場合は以降の処理をスキップ
         # raise StopIteration("テストデータがありません") # またはここで終了
    except Exception as collect_err:
         print(f"予測収集中にエラーが発生しました: {collect_err}")
         raise collect_err

    # 予測が収集できた場合のみ混同行列を計算・表示
    if true_labels and pred_labels:
        print("予測の収集が完了しました。混同行列を計算・表示します。")

        # 混同行列の計算
        try:
            cm = confusion_matrix(true_labels, pred_labels)
        except ValueError as cm_err:
             print(f"混同行列の計算中にエラー: {cm_err}")
             # ラベルの不一致などが考えられる
             raise cm_err

        # 混同行列の可視化
        plt.figure(figsize=(8, 6)) # サイズを少し調整
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                    xticklabels=class_names, yticklabels=class_names)
        plt.xlabel('Predicted Label') # ラベル名を修正
        plt.ylabel('True Label')     # ラベル名を修正
        plt.title('Confusion Matrix')
        plt.show()

        # 詳細な分類レポートを表示
        print("\n分類レポート:")
        try:
            # zero_division=0 を追加して、ゼロ除算が発生した場合の警告を抑制し、値を0にする
            report = classification_report(true_labels, pred_labels, target_names=class_names, digits=4, zero_division=0)
            print(report)
        except ValueError as report_err:
             print(f"分類レポートの生成中にエラー: {report_err}")
             # ラベルの不一致などが考えられる
             raise report_err
    else:
        print("予測データが収集されなかったため、混同行列の計算と表示をスキップしました。")


# NameError は model, data_module, config_path, config が未定義の場合
# FileNotFoundError は設定ファイルが見つからない場合（configロード時）
# KeyError は config 辞書に必要なキーがない場合
# ImportError はモジュールインポート失敗時
# StopIteration はデータローダーが空の場合
except (NameError, FileNotFoundError, KeyError, ImportError, StopIteration) as e:
     print(f"混同行列の計算に必要な変数、設定、またはデータが見つかりません: {e}")
# 広範な例外捕捉は避ける
except Exception as e: # より具体的な例外を捕捉することが望ましい
    print(f"混同行列の計算または表示中に予期せぬエラーが発生しました: {e}")
    import traceback
    traceback.print_exc()